In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle

In [29]:
users = pd.read_csv('data/full/train.csv')

In [30]:
users = users[['user_id','target']]

In [31]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [32]:
tweets = tweets.drop_duplicates()

In [33]:
tweets = pd.merge(tweets, users, on='user_id')

In [34]:
tweets.drop(['user_id'], axis=1, inplace=True)

In [35]:
def remove_rt(x):
    if 'RT' in x:
        x = x.replace('RT', '')
        try:
            return x[x.rindex(':')+2:]
        except:
            return x
    else:
        return x

In [36]:
#def translate(x):
#    try:
#        return translator.translate(x).text
#    except:
#        return x

In [37]:
stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]

In [39]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: remove_rt(x))

In [40]:
# tweets['full_text'] = tweets['full_text'].apply(lambda x: translate(x))

In [41]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: re.sub(r'^\/t.co\/[^\s]+', '', x))

In [42]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))

In [43]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: x.lower())

In [44]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: remove_stop(x))

In [45]:
tweets['full_text'] = tweets['full_text'].astype(str)

In [49]:
tweets.full_text[8] == ''

False

In [50]:
tweets = tweets[tweets.full_text!='[]']

In [51]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))


stem_vectorizer = StemmedCountVectorizer(stemmer)

pipeline = Pipeline([('vect', stem_vectorizer), 
                     ('tfidf', TfidfTransformer()), 
                     ('mnb', MultinomialNB(fit_prior=False))])
X_train, X_test, y_train, y_test = train_test_split(tweets.full_text, tweets.target, test_size=0.33, random_state=42)

clf = pipeline.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('accuracy: ' + str(np.mean(y_pred == y_test)))
print('partial f1: ' + str(f1_score(y_test, y_pred, average=None)))
print('total f1: ' + str(f1_score(y_test, y_pred, average='macro')))

accuracy: 0.7212919451982263
partial f1: [0.55321851 0.80013627 0.80739234 0.46812603 0.54103089]
total f1: 0.6339808083113663


accuracy: 0.7212919451982263
partial f1: [0.55321851 0.80013627 0.80739234 0.46812603 0.54103089]
total f1: 0.6339808083113663

In [54]:
clf.predict_proba(pd.Series("Try this awesome app"))

array([[0.05384182, 0.14878841, 0.56440358, 0.01245457, 0.22051163]])

In [52]:
pickle.dump( clf, open( "nb.model", "wb" ) )